In [1]:
using Revise
include("EFITModule/EFIT.jl")
include("EFITModule/ARBSCATT.jl")

using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie
ParallelStencil.@reset_parallel_stencil()

USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [2]:
matGrid, params = Main.ARBSCATT.loadARBSCATT("TestFiles");
transducer = Main.ARBSCATT.loadTransducers("TestFiles")

1-element Vector{Main.ARBSCATT.Transducer}:
 Main.ARBSCATT.Transducer(715, 358, 866, 45, 3001, 0.0f0, 0.0f0, Float32[0.0, 290181.2, 555390.4, 772804.56, 923713.9, 995131.56, 980911.7, 882277.94, 707718.4, 472255.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [3]:
params

Dict{Any, Any} with 10 entries:
  "nz"      => 1431
  "nx"      => 867
  "μ"       => 8.09365f10
  "maxT"    => 3001.0
  "λ"       => 1.05603f11
  "ρ"       => 7710.0
  "ny"      => 716
  "dt"      => 2.08257f-8
  "outputN" => 10.0
  "ds"      => 0.000212459

In [4]:
mat = Main.EFIT.IsoMat(params["λ"],params["μ"],params["ρ"],params["λ"] + 2*params["μ"])

Main.EFIT.IsoMat(7710.0f0, 1.056031f11, 8.093649f10, 2.6747607f11)

In [9]:
materials = [Main.EFIT.IsoMats["LightweightGeneric"],mat]



dx = params["ds"]
dt = params["dt"]


grid = Main.EFIT.EFITGrid(matGrid.+1,materials,dt,dx);


Creating grid of size 867, 716, 1431



In [10]:
path = "/media/data/2022 Data/RailTest_2,5MHz_14_May"
touch("$path/test.txt")

"/media/data/2022 Data/RailTest_2,5MHz_14_May/test.txt"

In [12]:
fig,ax,plt = volume(Array(grid.vx[1:2:end,1:2:end,1:2:end]),algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)

In [13]:
sx = 866
sy = 358
sz = 715
r = 45

rx = sx:sx
ry = sy-r:sy+r
rz = sz-r:sz+r

nSteps = 3000
for i = 1:nSteps
    if i % 10 == 0
        println("Saving at $i of $nSteps")
        Main.EFIT.writeToBOV(i*dt,i,grid,directory=path)
        plt.volume = sqrt.(grid.vx[1:2:end,1:2:end,1:2:end].^2 .+ grid.vy[1:2:end,1:2:end,1:2:end].^2 .+ grid.vz[1:2:end,1:2:end,1:2:end].^2)
    end
    println("$i of $nSteps")
    Main.EFIT.SimStep!(grid)
    if i < transducer[1].drivelen
        drive = transducer[1].drivef[i]
    else
        drive = 0

    end
    @parallel (rx,ry,rz) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), drive)
end

1 of 500


2 of 500


3 of 500


4 of 500


5 of 500


...

24 of 500


25 of 500


26 of 500


27 of 500


28 of 500
